In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2
import tensorflow as tf
from tensorflow import keras
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.utils import normalize
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.utils import to_categorical

In [2]:
image_directory = "/kaggle/input/brain-tumor-detection/"
healthy_images = os.listdir(image_directory + 'no/')
tumor_images = os.listdir(image_directory + 'yes/')
dataset = []
label = []
inp_size = 64
# print(healthy_images)

In [3]:
# The interpreter should be able to read .jpg files by recognizing them
# Therefore, we msut split the image labels to into head and tail
# .split() returns a head, tail tuple
# enumerate() keeps a count of iterables

for i, image_name in enumerate(healthy_images):
    if(image_name.split('.')[1] == 'jpg'):
        image = cv2.imread(image_directory + 'no/' + image_name) #array
        image = Image.fromarray(image, 'RGB') # we obtain the image from the array
        image = image.resize((inp_size, inp_size)) #resizing images to 64x64
        
        # 1 for brain tumor
        # 0 for healthy
        # Therefore, we convert image to dependent and independent variables
        
        dataset.append(np.array(image)) # independent feature(s)
        label.append(0) # dependent feature
        

In [4]:
for i, image_name in enumerate(tumor_images):
    if(image_name.split('.')[1] == 'jpg'):
        image = cv2.imread(image_directory + 'yes/' + image_name)
        image = Image.fromarray(image)
        image = image.resize((inp_size, inp_size))
        
        dataset.append(np.array(image))
        label.append(1)

In [5]:
dataset = np.array(dataset)
label = np.array(label)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(dataset, label, test_size = 0.2, random_state = 0)

In [7]:
print(X_train.shape)

# 2400 images of 64x64 size with 3 channels ie RGB

(2400, 64, 64, 3)


Normalization of the dataset

In [8]:
X_train = normalize(X_train, axis = 1)
X_test = normalize(X_test, axis = 1)

#Convert y to categorical format
y_train = to_categorical(y_train, num_classes = 2)
y_test = to_categorical(y_test, num_classes = 2)

Model Creation

In [9]:
# depth of conv filter is equal to 

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape = (inp_size, inp_size, 3), padding = 'same')) # 32 is the number of filters (recommended to be power of 2)
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(32, (3, 3), kernel_initializer = 'he_uniform', padding = 'same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, (3, 3), kernel_initializer = 'he_uniform', padding = 'same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2)) # cross entropy Binary is 1 for categorical it is 2
model.add(Activation('softmax'))

In [10]:
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [11]:
model.fit(X_train, y_train, batch_size = 32, verbose = 1, epochs = 50, validation_data = (X_test, y_test), shuffle = False)

Epoch 1/50
75/75 [==============================] - 10s 11ms/step - loss: 0.5313 - accuracy: 0.7417 - val_loss: 0.4880 - val_accuracy: 0.7767
Epoch 2/50
75/75 [==============================] - 0s 6ms/step - loss: 0.4197 - accuracy: 0.8196 - val_loss: 0.3865 - val_accuracy: 0.8217
Epoch 3/50
75/75 [==============================] - 1s 7ms/step - loss: 0.3067 - accuracy: 0.8687 - val_loss: 0.3112 - val_accuracy: 0.8500
Epoch 4/50
75/75 [==============================] - 1s 7ms/step - loss: 0.2154 - accuracy: 0.9133 - val_loss: 0.2200 - val_accuracy: 0.9017
Epoch 5/50
75/75 [==============================] - 1s 7ms/step - loss: 0.1494 - accuracy: 0.9446 - val_loss: 0.1536 - val_accuracy: 0.9583
Epoch 6/50
75/75 [==============================] - 0s 6ms/step - loss: 0.1053 - accuracy: 0.9638 - val_loss: 0.1512 - val_accuracy: 0.9483
Epoch 7/50
75/75 [==============================] - 0s 6ms/step - loss: 0.0773 - accuracy: 0.9733 - val_loss: 0.1054 - val_accuracy: 0.9633
Epoch 8/50
75/75 [

In [12]:
model.save('BrainTumor_100_categorical.h5')

**Testing the model (binary)**

In [13]:
from keras.models import load_model

mode = load_model('/kaggle/input/models/BrainTumor_100.h5')

img = cv2.imread('/kaggle/input/brain-tumor-detection/pred/pred0.jpg')
img = Image.fromarray(img, 'RGB')
img = img.resize((64, 64))
img = np.array(img)

img = np.expand_dims(img, axis = 0)
res = np.argmax(model.predict(img), axis = -1)
print(res)

1/1 [==============================] - 0s 132ms/step
[0]


In [14]:
from keras.models import load_model

mode = load_model('/kaggle/input/models/BrainTumor_100.h5')

img = cv2.imread('/kaggle/input/brain-tumor-detection/pred/pred5.jpg')
img = Image.fromarray(img, 'RGB')
img = img.resize((64, 64))
img = np.array(img)

img = np.expand_dims(img, axis = 0)
res = np.argmax(model.predict(img), axis = -1)
print(res)

1/1 [==============================] - 0s 18ms/step
[1]


**Testing the model (binary)**

In [15]:
from keras.models import load_model

mode = load_model('/kaggle/input/models/BrainTumor_100_categorical.h5')

img = cv2.imread('/kaggle/input/brain-tumor-detection/pred/pred0.jpg')
img = Image.fromarray(img, 'RGB')
img = img.resize((64, 64))
img = np.array(img)

img = np.expand_dims(img, axis = 0)
res = np.argmax(model.predict(img), axis = -1)
print(res)

1/1 [==============================] - 0s 18ms/step
[0]


In [16]:
from keras.models import load_model

mode = load_model('/kaggle/input/models/BrainTumor_100_categorical.h5')

img = cv2.imread('/kaggle/input/brain-tumor-detection/pred/pred5.jpg')
img = Image.fromarray(img, 'RGB')
img = img.resize((64, 64))
img = np.array(img)

img = np.expand_dims(img, axis = 0)
res = np.argmax(model.predict(img), axis = -1)
print(res)

1/1 [==============================] - 0s 18ms/step
[1]
